In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
!cd '/content/gdrive/My Drive/Object_Detection_Transformers/main-repo/'

In [ ]:
import numpy as np
import os

from tensorflow import keras
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import cv2


In [ ]:
def bbox_iou(bbox_a, bbox_b):
    """Calculate the Intersection of Unions (IoUs) between bounding boxes.

    IoU is calculated as a ratio of area of the intersection
    and area of the union.

    both inputs and output should be python list
    """
    ##########--WRITE YOUR CODE HERE--##########

    a = np.array(bbox_a)
    b = np.array(bbox_b)
    a = a.repeat(b.shape[0], axis = 0)

    top_left_a = a[ : , [0,1]] 
    if(b.shape[0] == 0):
      return [[0]]
    top_left_b = b[ : , [0,1]]
    bottom_right_a = a[ : , [2,3]] 
    bottom_right_b = b[ : , [2,3]]
    max_pts = np.maximum(top_left_a , top_left_b)
    min_pts = np.minimum(bottom_right_a , bottom_right_b)

    inter_pts = np.concatenate((max_pts , min_pts) , axis = 1)

    inter_dim = np.stack( [inter_pts[: , 2] -  inter_pts[: , 0] , inter_pts[: , 3] -  inter_pts[: , 1] ] , axis = 0).T
    inter_dim[inter_dim < 0] = 0 
    inter = np.multiply( inter_dim[:, 0] , inter_dim[:,1])

    gt_area = (bbox_a[0][2] - bbox_a[0][0]) * (bbox_a[0][3] - bbox_a[0][1])

    bbox_b_dim = np.stack( [b[: , 2] -  b[: , 0] , b[: , 3] -  b[: , 1] ] , axis = 0).T
    bbox_b_areas = np.multiply( bbox_b_dim[:, 0] , bbox_b_dim[:,1])

    union = bbox_b_areas - inter + gt_area

    iou = [( inter / union ).tolist()]



    ##########-------END OF CODE-------########## 
    return iou


In [ ]:
def get_unique_boxes(boxes):

  if(boxes == []):
    return []

  boxes_unique = [boxes.pop()]

  for box in boxes:
    iou = bbox_iou([box] , boxes_unique)[0]
    drop = False
    for val in iou:
      if(val > 0.60):
        drop = True
        break
    if(not drop):
      boxes_unique.append(box)
          
  return boxes_unique




In [ ]:
def calculate_iou(gt , pred):
  total = 0
  count = 0
  for box in gt:
    iou = bbox_iou([box], pred)[0]
    max_iou = max(iou)
    idx = iou.index(max_iou)
    if(pred == []):
      break
    pred.pop(idx)
    total = total + max_iou
    count = count + 1
  return total/count



In [ ]:



images, targets = [], []

#------------------TESTING DATA-------------------------------
path_images = "/content/gdrive/My Drive/Object_Detection_Transformers/main-repo/dataset/test/images/"
path_annot = "/content/gdrive/My Drive/Object_Detection_Transformers/main-repo/dataset/test/labels/"
path_result = "/content/gdrive/My Drive/Object_Detection_Transformers/main-repo/results/labels/"


# list of paths to images and annotations
image_paths = [
    f for f in os.listdir(path_images) if os.path.isfile(os.path.join(path_images, f))
]
annot_paths = [
    f for f in os.listdir(path_annot) if os.path.isfile(os.path.join(path_annot, f))
]
image_result = [
    f for f in os.listdir(path_result) if os.path.isfile(os.path.join(path_result, f))
]
image_size = 512  

image_paths.sort()
annot_paths.sort()
image_result.sort()
horn={'0':"Elk",'1':"Stag"}
total = 0
misclassified = 0
idx=[]
for i in range(0 , 215): #[146, 151] 
    print(image_paths[i])
    
    image = keras.utils.load_img(
        path_images + image_paths[i],
    )
    (w, h) = image.size[:2]
    # resize train set images
    #if i < int(len(annot_paths) * 0.8):
        # resize image if it is for training datase t
    image = image.resize((image_size, image_size))
    
    # convert image to array and append to list
    images.append(keras.utils.img_to_array(image))
    
    # plt.figure(figsize=(15, 5))
    # plt.subplot(1, 2, 1)
    # plt.imshow(image)
    # plt.title(' test image')
    # plt.axis("off")
    fig, (ax1,ax2) = plt.subplots(1, 2, figsize=(10, 10))
    plt.axis('off')

    # Display the image
    ax1.imshow(image)
   
    f = open(path_annot +  image_paths[i][:-3]+"txt", "r")
    line = f.readline()
    coral_type = ""
    box_gt = []
    while line:
       coordinates = line.split()
       line = f.readline()
       x_center, y_center ,box_width, box_height  = float(coordinates[1]) , float(coordinates[2]) , float(coordinates[3]) , float(coordinates[4] )
       top_left_x = x_center - box_width / 2 
       top_left_y = y_center - box_height / 2
       bottom_right_x = x_center + box_width / 2 
       bottom_right_y = y_center + box_height / 2  
 
       #print(top_left_x * image_size , top_left_y * image_size,bottom_right_x * image_size, bottom_right_y * image_size)

       box = [top_left_x * image_size , top_left_y * image_size,bottom_right_x * image_size, bottom_right_y * image_size]
       box_gt.append(box)        
        # Create the bounding box
        
       rect = patches.Rectangle(
          (top_left_x * image_size, top_left_y * image_size),
          (bottom_right_x - top_left_x) * image_size,
          (bottom_right_y - top_left_y) * image_size,
          facecolor="none",
          edgecolor="red",
          linewidth=2,
          label='Label',
        )
       ax1.add_patch(rect)
       x= (top_left_x * image_size)+(bottom_right_x - top_left_x) * image_size/2
       y= (top_left_y * image_size)+(bottom_right_y - top_left_y) * image_size/2
       ax1.annotate(horn[coordinates[0]], (x, y), color='white', weight='bold', fontsize=10, ha='center', va='center')
       coral_type = coordinates[0]
       #plt.add_patch(rect)
    f.close()

    if(coral_type == ""):
      total = total + 1
      continue

    image1 = keras.utils.load_img(
        path_images + image_paths[i],
    )
    
    f = open(path_result +  image_paths[i][:-3]+"txt", "r")
    line = f.readline()
    boxes = []
    while line:
       coordinates = line.split()
       line = f.readline()
       x_center, y_center ,box_width, box_height  = float(coordinates[1]) , float(coordinates[2]) , float(coordinates[3]) , float(coordinates[4] )
       top_left_x = x_center - box_width / 2 
       top_left_y = y_center - box_height / 2
       bottom_right_x = x_center + box_width / 2 
       bottom_right_y = y_center + box_height / 2  
 
       box = [top_left_x * image_size , top_left_y * image_size,bottom_right_x * image_size, bottom_right_y * image_size]
       boxes.append(box)
    f.close()
        # Create the bounding box
    unique_boxes = get_unique_boxes(boxes)
    for box in unique_boxes:
       top_left_x  , top_left_y  , bottom_right_x , bottom_right_y  = box[0] , box[1] , box[2] , box[3]
       rect = patches.Rectangle(
          (top_left_x , top_left_y ),
          (bottom_right_x - top_left_x) ,
          (bottom_right_y - top_left_y) ,
          facecolor="none",
          edgecolor="limegreen",
          linewidth=2,
          label='Label',
        )
       ax2.add_patch(rect)
       x= (top_left_x ) + (bottom_right_x - top_left_x) * 1/2
       y= (top_left_y )+(bottom_right_y - top_left_y) * 1/2
       ax2.annotate(horn[str(int(float(coordinates[0])))], (x, y), color='white', weight='bold', fontsize=10, ha='center', va='center')
    if( int(coral_type) != int(float(coordinates[0]))):
      idx.append(i)
      misclassified = misclassified + 1

    ax2.imshow(image1)
    plt.show()
    iou = calculate_iou(box_gt, unique_boxes)
    print("iou" , " = " , iou)
    total = total + iou
    # Add the bounding box to the image
print("IOU = " , total / 215)  
print(idx)

Output hidden; open in https://colab.research.google.com to view.